In [1]:
import os
import json
from pybtex.database.input import bibtex

#WHAT TO DO NEXT
#Invited seminars module
#figure out how to handle number-limited section


In [17]:
#Presentation function
def presentation(block):
    string = ""
    for i,author in enumerate(block["authors"]):
        if (i+1) == block["speaker"]:
            string += "{\it "+author+"}"
        else:
            string += author
        if (i+1) == len(block["authors"]):
            string += ". "
        elif (i+1) == len(block["authors"])-1:
            string += ", and "
        else:
            string += ", "
    
    string += block["title"]+". "
    string += str(block["year"])+" "+block["conference"]+', '
    string += block["address"]+". "
    if "invited" in block and block["invited"]:
        string += "{\\it Invited}."
    if "note" in block:
        string += "{\\it "+block["note"]+"}."
    
    return string

def presentations_section(pres_data):
    output = ""
    presentations = json.load(open(pres_data,mode="r"))
    for entry in presentations:
        #print(entry['title'])
        output += presentation(entry)
        output += "\n\n"
        #output
    return output

def publication(bibtex_data,id):
    #block = bibdata.entries[bib_id].fields
    block = bibtex_data.entries[id].fields
    authors = bibtex_data.entries[id].persons["author"]
    string = ""
    num_authors = len(authors)
    for i, author in enumerate(authors):
        #string += author.last_name[0]+", "
        #string += author.first_names[0]
        string += str(author)
        if (i == num_authors-2):
            string += " and "
        else:
            string += ", "
    string += block["title"]+". "
    string += block["year"]+". "
    string += "{\it "+block["journal"]+"}"
    try:
        string += ", {\\bf "+block["volume"]+"}, "
        page = block["pages"].split('--')[0]
        string += page+"."
    except:
        string += "."
    #print("\\url{https://doi.org/"+block["doi"]+"}")
    # Attempt to add a DOI link
    try:
        string += " "+"\\url{https://doi.org/"+block["doi"]+"}"
    except:
        print("failed", block["title"])
        string += " "+block["note"]
        pass
    return string

def publications_section(pubs_data,reverse=True):
    output = ""
    #open a bibtex file
    parser = bibtex.Parser()
    bibdata = parser.parse_file(pubs_data)

    #create a pointer array that re-orders the BibTeX entries into publication order
    pubs_order = ['' for x in range(len(bibdata.entries))]
    for (i,bib_id) in enumerate(bibdata.entries):
        b = bibdata.entries[bib_id].fields
        pubs_order[int(b["order"])-1] = bib_id
    
    #loop through the individual references
    if reverse:
        order = reversed(pubs_order)
    else:
        order = pubs_order
    for bib_id in order:
        output += publication(bibdata,bib_id)
        output += "\n\n"
    
    return output

def contact_info(data):
    output="\\section{\sc Contact Information}"
    output += '\n \\vspace{.05in}'    
    output += '\\begin{tabular}{@{}p{2in}p{0.5in}p{2in}}'
    output += data["Address"][0]
    output += '   &   {\it Phone:}  & '
    output += data["Phone"]
    output += ' \\\\\n'  #this is the new line indicator in tabular mode
    output += data["Address"][1]
    output += '   &   {\it Email:}  & '
    output += data["Email"]
    output += '\\end{tabular}'
    
    return output

class latex_CV:
    # Constructor method
    def __init__(self):
        self.master = ''
        self.add_TeX_block('\\documentclass[margin,line]{res}')
        self.whitespace()
    
    # Overload the returned value of self
    def __str__(self):
        return self.master
    
    # Method to add a carriage return to the string
    def newline(self):
        self.master += '\n'
        
    # Method to add a line of whitespace
    def whitespace(self):
        self.master += '\n\n'
        
    # Method to finalize the document
    def finalize(self):
        self.add_TeX_block('\\end{resume}\n')
        self.add_TeX_block('\\end{document}')

    # Method to insert TeX from a separate file
    def add_TeX_file(self,file):
        with open(file,mode='r') as f:
            new_text = f.read()
        self.newline()
        self.master += new_text
        self.newline()

    # Method to add a block of TeX
    def add_TeX_block(self,block):
        self.newline()
        self.master += block
        self.newline()
        
    # Method to write the current buffer to disk
    def dump(self):
        with open('cv.tex',mode='w') as f:
            f.write(self.master)
    
    # Method to compile the document
    def compile(self):
        with open('cv.tex',mode='w') as f:
            f.write(self.master)
        os.system('pdflatex cv')
        os.system('pdflatex cv')
#         for x in ['tex','aux','log']:
#             try:
#                 os.remove('cv.'+x)
#             except:
#                 continue
        
# Instantiation
test = latex_CV()

test.add_TeX_file('preamble.tex')
test.add_TeX_block('\\begin{document}\n')
test.add_TeX_block('\\name{Daniel William Siderius\\vspace*{.1in}}\n')
test.add_TeX_block('\\begin{resume}\n')
test.add_TeX_file('header.tex')

test.add_TeX_block('\section{\sc Education}\n')
test.add_TeX_file('education.tex')

test.add_TeX_block('\section{\sc Intellectual Property}\n')
test.add_TeX_file('IP.tex')

test.add_TeX_block('\section{\sc Publications}\n')
test.add_TeX_block(publications_section("publications.bib",reverse=True))
test.newline()

test.add_TeX_block('\section{\sc Conference Presentations}\n')
test.add_TeX_block(presentations_section('presentations.json'))

test.add_TeX_block('\section{\sc Invited Seminars}\n')
test.add_TeX_block(presentations_section('seminars.json'))


test.finalize()

#print(test)
test.dump()

test.compile()


This is pdfTeX, Version 3.141592653-2.6-1.40.22 (TeX Live 2021) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
(./cv.tex
LaTeX2e <2020-10-01> patch level 4
L3 programming layer <2021-02-18> (./res.cls
Document Style `res' <26 Sep 89>.
Document Class: res 2000/05/19 v1.4b Resume class
(/usr/local/texlive/2021/texmf-dist/tex/latex/base/article.cls
Document Class: article 2020/04/10 v1.4m Standard LaTeX document class
(/usr/local/texlive/2021/texmf-dist/tex/latex/base/size10.clo))
No auxiliary output files.

) (/usr/local/texlive/2021/texmf-dist/tex/latex/fancyhdr/fancyhdr.sty)
(/usr/local/texlive/2021/texmf-dist/tex/latex/base/ifthen.sty)
(/usr/local/texlive/2021/texmf-dist/tex/latex/hyperref/hyperref.sty
(/usr/local/texlive/2021/texmf-dist/tex/generic/ltxcmds/ltxcmds.sty)
(/usr/local/texlive/2021/texmf-dist/tex/generic/iftex/iftex.sty)
(/usr/local/texlive/2021/texmf-dist/tex/generic/pdftexcmds/pdftexcmds.sty
(/usr/local/texlive/2021/texmf-dist/tex/gener

) (/usr/local/texlive/2021/texmf-dist/tex/latex/fancyhdr/fancyhdr.sty)
(/usr/local/texlive/2021/texmf-dist/tex/latex/base/ifthen.sty)
(/usr/local/texlive/2021/texmf-dist/tex/latex/hyperref/hyperref.sty
(/usr/local/texlive/2021/texmf-dist/tex/generic/ltxcmds/ltxcmds.sty)
(/usr/local/texlive/2021/texmf-dist/tex/generic/iftex/iftex.sty)
(/usr/local/texlive/2021/texmf-dist/tex/generic/pdftexcmds/pdftexcmds.sty
(/usr/local/texlive/2021/texmf-dist/tex/generic/infwarerr/infwarerr.sty))
(/usr/local/texlive/2021/texmf-dist/tex/latex/graphics/keyval.sty)
(/usr/local/texlive/2021/texmf-dist/tex/generic/kvsetkeys/kvsetkeys.sty)
(/usr/local/texlive/2021/texmf-dist/tex/generic/kvdefinekeys/kvdefinekeys.sty)
(/usr/local/texlive/2021/texmf-dist/tex/generic/pdfescape/pdfescape.sty)
(/usr/local/texlive/2021/texmf-dist/tex/latex/hycolor/hycolor.sty)
(/usr/local/texlive/2021/texmf-dist/tex/latex/letltxmacro/letltxmacro.sty)
(/usr/local/texlive/2021/texmf-dist/tex/latex/auxhook/auxhook.sty)
(/usr/local/tex

In [ ]:

#Multiple Blocks via the "Sequence" entry
design_data = json.load(open("design.json",mode='r'))

newcv = latex_CV()

newcv.add_TeX_file('preamble.tex')
newcv.add_TeX_block('\\begin{document}\n')

# Name
newcv.add_TeX_block('\\name{'
                    +design_data["name"]["given"]+' '
                    +design_data["name"]["middle"]+' '
                    +design_data["name"]["family"]
                    +'\\vspace*{.1in}}\n')

newcv.add_TeX_block('\\begin{resume}\n')
newcv.add_TeX_file('header.tex')

newcv.add_TeX_block(contact_info(design_data["ContactInformation"]))

newcv.add_TeX_block('\section{\sc Education}\n')
newcv.add_TeX_file('education.tex')

print(design_data["Sequence"])

for section in design_data["Sequence"]:
    #Extract the current section
    data_block = design_data[section]
    if isinstance(data_block["number"],int):
        how_many = data_block["number"]
    else:
        how_many = 1e6 #I don't think I'll have more than 1 million of any category
    #Write the current section
    newcv.add_TeX_block("\section{\sc "+data_block["label"]+'}\n')
    if data_block["label"] == "Publications":
        newcv.add_TeX_block(publications_section(data_block["file"]))
        #continue
    elif data_block["label"] == "Conference Presentations":
        newcv.add_TeX_block(presentations_section(data_block["file"]))
        #continue
    elif data_block["label"] == "Invited Seminars":
        #newcv.add_TeX_block("WRITE THIS MODULE")
        newcv.add_TeX_block(presentations_section(data_block["file"]))
    else:
        newcv.add_TeX_file(data_block["file"])
#     else:
#         if how_many < 1e6:
#             output.write(str(how_many)+" Most Recent "+data_block["label"]+"\n\n")
#         input_data = open(data_block["file"])
#         items = []
#         for line in input_data:
#             if line != '\n':
#                 items.append(line.strip('\n'))
#         input_data.close()
#         for (count,item) in enumerate(items):
#             if count < how_many:
#                 #print item
#                 output.write(item+'\n'+'\n')


newcv.finalize()
#print(newcv)
newcv.dump()
newcv.compile()

In [3]:
print(design_data["ContactInformation"]["Address"][0])

NameError: name 'design_data' is not defined